# U.T.6 Redes Neuronales.
## Redes convolucionales (CNN)
### Introducción
Las CNN son muy populares al poder conseguir resultados muy interesantes en el reconocimiento de imágenes.

Son capaces de aprender características de una imagen por sí solas a partir de los datos.

Reconocen patrones locales en pequeñas partes de la imagen (en la imagen en una zona de 5x5) en vez de aprenderlos en
toda la imagen.

Las redes suponen que la entrada será una imagen, con lo que la dimensión de los datos será un tensor 3D: alto, ancho
y profundidad (o canales de color).

![](img/ut06_25.png)

Cada capa abstrae o reconoce un nivel diferente, en la primera parecerán bordes, en la segunda formas simples, en los
siguientes patrones más complejos

![](img/ut06_26.png)

Las CNN realizan su trabajo muy bien por dos ideas fundamentales de las mismas:
- Escasa conectividad. Un mapa de características está conectado un pequeño grupo de píxeles. Además, estos patrones
son invariantes a las traslaciones.
- Compartición de parámetros. Los mismos pesos se comparten entre diferentes grupos de píxeles.

Y estas ideas funcionan porque generalmente en una imagen los píxeles cercanos tendrán información relacionada entre
ellos en vez de aquellos que están mas alejados. Además, las imágenes se forman por jerarquías de patrones, que se van
reconociendo según vamos avanzando por las capas convolucionales.

![](img/ut06_27.png)

In [2]:
from keras import layers
from keras import models
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

# LAS NUEVAS CAPAS
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
print(model.summary())

313/313 [==============================] - 1s 3ms/step - loss: 0.0239 - accuracy: 0.9929

In [3]:
# CAPAS YA CONOCIDAS
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [4]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape)  # (60000, 28, 28)  solo un valor, monocromo
# convertir a la convención example,height,width,channel
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255  # normalizar
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

(60000, 28, 28)


In [5]:
# necesitamos que los labels con one-hot encode
print(train_labels[0])  # 5
train_labels = to_categorical(train_labels)

5


In [6]:
print(train_labels[0])  # [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64, verbose=0)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0303 - accuracy: 0.9902

Generalmente las CNN están compuestas por varias capas convolucionales y capas de submuestreo, seguidas por una o
más capas densas al final. Las capas de submuestreo (pooling) no tienen parámetros a entrenar, ni tienen pesos ni valor
de varianza, mientras que las capas convolucionales sí.

Las capa inicial convolucional necesita de entrada (height, width, channels) en el ejemplo anterior:
input_shape=(28, 28, 1).

La salida de cada capa convolucional y maxpoling es un tensor de dimension 3D
(height, width, channels_fiters)  (conv2d (Conv2D)    (None, 26, 26, 32)).  El número de canales o filtros.

El número de canales de salida se controla con el primer parámetro de las capas (32, o 64)[ layers.Conv2D(32,..]

La última capa convolucional sirve de entrada a una capa densa con el mismo número de entradas, por lo que hay que
aplanarla antes con flatter.

El tamaño de entrada se reduce por el problema de padding, si queremos el mismo debemos ajustarlo en la creación de
la capa.

La capa de salida se define con la función Softmax y con el número de clases a predecir.

La operación de Convolución
Cada capa de convolución aprende patrones locales dentro de la imagen en pequeñas ventanas de dos dimensiones,
no en toda la imagen.

El propósito principal de una capa es detectar rasgos visuales. Una vez aprenda una característica será capaz de
reconocerla en cualquier parte de la imagen.

Una capa es capaz de reconocer jerarquías de características utilizando lo aprendido por las capas superiores.
Estas jerarquías serán objetos, disposiciones de objetos, etc. Por lo que las CNN son capaces de aprender patrones
visuales complejos y de forma eficiente

La operación de convolución opera sobre tensores en 3D exclusivamente en los datos de entrada, con lo que será imprescindible esta transformación antes de introducir los datos

train_images = train_images.reshape((60000, 28, 28, 1))

Los tensores expresan alto, ancho y número de canales de color (1 para monocromo y tres o cuatro para color).
Como vemos a la red se le proporciona una dimensión más con los ejemplos

las etiquetas multiclase deben estar codificadas con el mecanismo one-hot (una etiqueta por columna)

train_labels = to_categorical(train_labels)

### La operación de Convolución
La capa CNN es muy diferente a la densa estudiada anteriormente, ya que esta conecta cada neurona con una parte
muy pequeña de la imagen (3x3 o 5x5) en vez de toda la imagen (esa ventana se denomina kernel).

![](img/ut06_28.png)

Como vemos en la imagen, la matriz de 5x5 se conectará con la primera neurona de la siguiente capa, si desplazamos
esta matriz una posición, se conectará con la segunda neurona, así hasta terminar la fila. Una vez finalizada se
desplaza la matriz una fila y a la primera columna y empezamos para la segunda fila, de esta manera se recorrerá toda
la imagen para conectar la segunda capa a la primera).

Este movimiento de desplazamiento genera dos problemas, el primero que la dimensionalidad de la capa de salida será
menor que la de entrada (se gestiona con el **padding**) y segundo, la cantidad de desplazamiento de la ventana puede ser
variable, en el ejemplo ha sido uno, pero se gestiona con el **stride**.

![](img/ut06_29.png)

Como cada neurona está conectada con un grupo pequeño de neuronas de la capa anterior (5x5 en el ejemplo) solo tendrá
una matriz de pesos de ese tamaño (25 elementos), esta matriz es compartida por toda la capa y se ajustará durante el
aprendizaje. Es decir, se usa la misma matriz para hacer todo el recorrido de la imagen

![](img/ut06_30.png)

![](img/ut06_31.png)

El **padding** es la opción de agregar ceros alrededor para no perder dimensionalidad.

El **stride** es el desplazamiento del kernel para recorrer la imagen.

Para una entrada de 5x5 con un kernel de 3x3 y un stride de (1,1) se genera un filtro de 3x3.

![](img/ut06_30.png)

#### Ejemplo

![](img/ut06_32.png)

En el ejemplo el padding es cero y que en cada operación el vector w se desplaza dos celdas a la derecha (stride=2,2).
Estos dos parámetros (p y s) tendremos que ajustarlos en nuestras capas convolucionales. El desplazamiento que debe
sufrir el kernel en cada operación se denomina stride y es 1,

La matriz de salida de la operación se denomina filtro, y es el resultado de mover el kernel por toda la imagen,
multiplicar las dos matrices de forma escalar y sumar los resultados

El problema es que cada filtro solo reconoce una característica, con lo que es necesario repetir este procedimiento
varias veces (32, 64) para la misma capa para que tenga en cuenta varias características, el número de filtros a definir
en una capa se establecen al crear la capa.

model.add(layers.Conv2D(32, (3, 3), activation='relu'),
  input_shape=(28, 28, 1))

El conjunto de filtros de una capa se denomina espacio de características o **feature space**.

![](img/ut06_33.png)

#### Consideraciones finales
Para trabajar con las redes convolucionales tendremos las siguientes características
- Imagen de entrada con una dimensión alto, ancho, canales
- A La imagen se le aplicará un kernel de tamaño 3x3 o 5x5
- Se le podrá añadir un número de columnas y filas de ceros en los extremos para evitar la reducción de la dimensión
de salida, se llama padding.
- El kernel se trasladará a lo largo de toda la imagen con saltos de stride, generalmente igual a 1 en todas las
direcciones
- Se multiplicará el valor de cada celda de la imagen con la correspondiente del kernel que cae encima y se sumará el
valor de todas las celdas resultantes
- Se repetirla el paso anterior hasta cubrir toda la imagen dando otro vector de salida llamado filtro
- El conjunto de todos los filtros es el espacio de características

### La Operación de Submuestreo (pooling)
El submuestreo realizan una simplificación de la información recogida por la capa de convolución y crea una versión
condesada de la información de la misma reduciendo la dimensionalidad. Se deben aplicar justo después de cada capa
de convolución.

![](img/ut06_34.png)

model.add(layers.MaxPooling2D((2, 2)))

El submuestreo, adopta dos formas: max-min o mean. En la primera opción se elige el mayor de todos los implicados y
en la segunda se hace la media.

El funcionamiento es similar, se establece un tamaño de kernel que se aplica a la entrada, se desplaza a través de
todos los valores y se va calculando el resultado final.

La salida será un vector de menor tamaño que la entrada en función de la dimensión del kernel (pooling size).

![](img/ut06_35.png)

Como la capa convolucional tiene varios filtros (32, 64) el pooling se aplicará a cada uno de ellos generando tantas
salidas como filtros haya.

![](img/ut06_36.png)

### Imágenes a color
Si trabajamos con imágenes en gris solo existirá una matriz con valores, pero si estamos usando colores estas aumentas
hasta 3 como mínimo (RGB) o incluso cuatro (RGBA) siendo la última la capa de transparencia.

En caso de usar imágenes a color, el procedimiento es el mismo, lo único que se repetirá tantas veces como canales
de entrada tengamos (RGB) y se sumarán las tres capas usando una matriz de suma que determine el modo que se pondera
dicha suma. Cada capa tiene su propio kernel.

El tratamiento de imágenes hace que se necesita gran cantidad de memoria y de cálculo, por lo que deberemos cargar
siempre nuestras imágenes como uint8.

### Hiperparámetros: Tamaño y número de filtros
El tamaño del kernel determinará el tamaño del filtro, a mayor tamaño más información se perderá. Los tamaños
recomendados son 3x3 y 5x5, pudiendo aumentar si la dimensionalidad de la imagen de entrada es muy grande.

Del mismo modo el número de filtros determinan el número de características a usar, estos valores suelen ser 32 o 64,
pero se podrá aumentar o disminuir en función de la complejidad del problema.

### Hiperparámetros: Padding
El pading es el número de columnas y filas rellenas de ceros que se aumentará el tensor inicial para mantener la
dimensión en el tensor de salida. Recordemos que al aplicar un filtro de tamaño mayor de 1x1 la salida reducirá su
dimensionalidad.

![](img/ut06_37.png)

![](img/ut06_38.png)

Los valores que podemos usar en el parámetro padding son:
- Full. p = m-1, generalmente no se usa.
- Same. Asegura que el vector de salida tiene el mismo tamaño que el de entrada. Es la opción más usada.
- Finally o Valid. Hace que p=0 y ser reduzca el tamaño del vector de salida, es el defecto en Keras.

### Hiperparámetros: Stride
Es el número de pasos que se desplaza el kernel hacia la izquierda y hacia abajo en el desplazamiento por la imagen.
Este valor suele ser una matriz indicando los pasos en cada sentido, pero por norma general no se utiliza y se usa
por defecto la matriz (1,1).

### Hiperparámetros: Regularización: Dropout y L1, L2
Generalmente se usa un porcentaje del 50% en las capas Dropout. El efecto es forzar a la capa a aprender patrones
más generales y robustos, mejorando además el sobreajuste. Hay que tener cuidado que no se deben poner tras la capa
de entrada.

In [ ]:
dropout_layer = keras.layers.Dropout(0.5)

conv_layer = keras.layers.Conv2D( filters=16, kernel_size=(3, 3), kernel_regularizer=keras.regularizers.l1(0.001))

fc_layer = keras.layers.Dense( units=16, kernel_regularizer=keras.regularizers.l2(0.001))

### Hiperparámetros: BatchNormalization
Esta regularización tiene el mismo efecto que el estudiado en el curso con la normalización, lleva las entradas
dentro de una media de cero y una desviación de 1. Estas capas aparecen justo antes de la capa Dropout y después de
una Densa o convolucional.

In [ ]:
dropout_layer = keras.layers.BatchNormalitation()

### Hiperparámetros: Regularización: Decaimiento del ratio de aprendizaje
Las redes CNN también se aprovechan de ellas.

In [ ]:
# opción uno
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(.. optimizer=optimizer)

# opción dos
reduce_lr = LearningRateScheduler(lambda x: 1e-1*0.9**x)
model.fit(train_images, train_labels, epochs=5, batch_size=64,
	callbacks=[reduce_lr])

### Hiperparámetros: Resumen
-Conv2D: tf.keras.layers.Conv2D
    -Filters. Número de características a encontrar. Suelen ir creciendo en orden de múltiplos de dos hasta las capas densas.
    -kernel_size. El tamaño es muy importante y no se debe usar grandes tamaños (5x5 o más) en cada capa, la única excepción es la primera capa que se suele utilizar con un tamaño de 5x5 y un stride de dos.
    -strides, depende de la imagen, si son pequeñas (64x64) a cero
    -padding, generalmente a SAME.
-MaxPool2D: tf.keras.layers.MaxPool2D
    -pool_size (2 para hace que se reduzca a la mitad el tamaño)
    -strides. No se suele establecer.
    -Padding. No se suele establecer.
-Normalización: tf.keras.layers.Normalization(), opcional
-Dropout: tf.keras.layers.Dropout2D
    -Rate (hasta 50%).

In [ ]:
model = keras.models.Sequential([
	keras.layers.Conv2D(64, 7, activation="relu", padding="same",
	input_shape=[28, 28, 1]),
	keras.layers.BatchNormalization(),
	keras.layers.MaxPooling2D(2),
	keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
	keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
	keras.layers.MaxPooling2D(2),
	keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
	keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
	keras.layers.MaxPooling2D(2),
	keras.layers.Flatten(),
	keras.layers.Dense(128, activation="relu"),
	keras.layers.Dropout(0.5),
	keras.layers.Dense(64, activation="relu"),
	keras.layers.Dropout(0.5),
	keras.layers.Dense(10, activation="softmax")
])

### Transferencia del aprendizaje
Hay gran cantidad de modelos para el reconocimiento cada uno de ellos con sus ventajas y sus inconvenientes

La idea principal es aprovechar lo que otros ya han creado para nuestro beneficio. Existen las siguientes técnicas
- Transformación de imágenes
- Uso de modelos ya entrenados
- Extracción de características
- Afinado de modelos

#### Transferencia del aprendizaje: Transformación de imágenes
Se pretende generar más datos de entrenamiento a partir de nuestros datos aplicando transformaciones aleatorias a la
imagen de entrada, produciendo otras similares, pero creibles

No generar imágenes que nunca podrían encontrase en realidad

Estas transformaciones las realizazá la clase ImageGenerator al vuelo, sin necesidad de almacenarlas en disco

In [ ]:
for fn in os.listdir(train_cats_dir):
    path = os.path.join(train_cats_dir, fn)
    img = load_img(path)
    data = img_to_array(img)
    samples = expand_dims(data, 0)

    # example of "rotation_range"
    datagen = ImageDataGenerator(rotation_range=45)

    it = datagen.flow(samples, batch_size=1)
    for i in range(6):
        plt.subplot(230 + 1 + i)
        batch = it.next()
        image = batch[0].astype('uint8')
        plt.imshow(image)
plt.show()

Vamos a transformar la imagen que se va a pasar a la red cada epoch de tal manera que la red nunca reciba
la misma imagen. Hay que recordar que en cada época se leen de nuevo todos los ejemplos que tenemos.

Multiplicaremos el número de imágenes por el número de épocas

Estos datos no se guardan, con lo que con cada ejecución variarán los datos de entrada

In [9]:
modelDA = Sequential()
modelDA.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
modelDA.add(MaxPooling2D(2, 2))
modelDA.add(Conv2D(64, (3, 3), activation='relu'))
modelDA.add(MaxPooling2D(2, 2))
modelDA.add(Conv2D(128, (3, 3), activation='relu'))
modelDA.add(MaxPooling2D(2, 2))
modelDA.add(Conv2D(128, (3, 3), activation='relu'))
modelDA.add(MaxPooling2D(2, 2))
modelDA.add(Flatten())
modelDA.add(Dense(512, activation='relu'))
modelDA.add(Dense(1, activation='sigmoid'))

modelDA.compile(loss='binary_crossentropy',
                optimizer=RMSprop(lr=1e-4),
                metrics=['acc'])
#

Creamos los transformadores
Solo se crean nuevas imágenes en las de train, nunca en los otros conjuntos, en los que simplemente se escalan y
normalizan como siempre con las imágenes.

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                       batch_size=20,
                       class_mode='binary',
                       target_size=(150, 150))
validation_generator = validation_datagen.flow_from_directory(
	 validation_dir, batch_size=20,
	 class_mode='binary',target_size=(150, 150))
test_generator = test_datagen.flow_from_directory(test_dir,
	 batch_size=20,class_mode='binary',
	 target_size=(150, 150))


Al trabajar con directorios y no con datos cargados, tenemos que crear una estructura de datos que vaya
proporcionándolos a la red según los mini-batch elegidos, en Python crearemos generadores sobre el contenido de
los directorios (flow_from_directory) a partir de los objetos generadores de imágenes

Por último, ya estamos en concidciones de entrenar y validar el modelo

In [ ]:
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size
historyDA = modelDA.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        verbose=2)

print(steps_per_epoch)
print(validation_steps)
test_lost, test_acc = modelDA.evaluate(test_generator)
print("Test Accuracy:", test_acc)

#### Transferencia del aprendizaje: Uso de modelos ya entrenados
En lugar de entrenar una red completamente, podemos usar una entrenada para nuestros fines. Hay dos aproximaciones
usarlo directamente, o modificar las capas de la red entrenada.

Es muy simple hacer uso de un modelo ya entrenado, simplemente lo cargamos, preparamos las imágenes al tamaño que
espera el modelo, procesamos dichas imágenes y realizamos la predicción.

In [ ]:
model = keras.applications.resnet50.ResNet50(weights="imagenet")
images_resized = tf.image.resize(images, [224, 224])
inputs = keras.applications.resnet50.preprocess_input(
		images_resized * 255)
Y_proba = model.predict(inputs)

#### Transferencia del aprendizaje: Extracción de características
En un modelo ya entrenado fijamos algunas capas y reentrenamos con nuestros ejemplos del problema en particular que
queremos resolver mejorando los tiempos de entrenamiento y el rendimiento final

Hay que recordar que las CNN aprenden de lo más simple a lo más complejo, con lo que si vamos eliminando capas desde
la salida, nos quedaremos con los datos que generalizan bien para todas las clases, en nuestro ejemplo de animales

En este tipo vamos a liberar el modelo del discriminante final, las capas densas (include_top=False)

Congelamos todas las capas del modelo entrenado, creamos nuestro clasificador y creamos el modelo nuevo compuesto de
ambos, para pasar a entrenarlo después

In [ ]:
pre_trained_model = VGG16(input_shape=(150, 150, 3),
                          include_top=False,
                          weights='imagenet')
pre_trained_model.summary()

modelFE = Sequential()
modelFE.add(pre_trained_model)
modelFE.add(Flatten())
modelFE.add(Dense(256, activation='relu'))
modelFE.add(Dense(1, activation='sigmoid'))  # para la clasificación binaria
modelFE.summary()

modelFE.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['acc'])

batch_size = 20
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

historyFE = modelFE.fit(train_generator,
        validation_data=validation_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        validation_steps=validation_steps,
        verbose=2)


#### Transferencia del aprendizaje: Afinado del modelo
Un ajuste más acertado generalmente es en el que además de eliminar el clasificador, no se fijan todas las capas del
modelo que quedan, solo las más altas, para que se ajusten también a nuestro problema las últimas capas
convolucionales.

Para que funcione correctamente, deberemos establecer ritmos de aprendizajes lentos para evitar que cambien mucho los
parámetros ya aprendidos en las capas no fijas convolucionales.

In [ ]:
pre_trained_model = VGG16(input_shape=(150, 150, 3),
                          include_top=False,
                          weights='imagenet')

pre_trained_model.trainable = True
set_trainable = False

for layer in pre_trained_model.layers:
    if layer.name == 'block5_conv1':  # solo el block_5 se entrenará
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
pre_trained_model.summary()

modelFT = Sequential()
modelFT.add(pre_trained_model)
modelFT.add(Flatten())
modelFT.add(Dense(256, activation='relu'))
modelFT.add(Dense(1, activation='sigmoid'))
modelFT.summary()
modelFT.compile(loss='binary_crossentropy',
                optimizer=RMSprop(lr=1e-4),
                metrics=['acc'])
historyFT = modelFT.fit(
        train_generator,
        validation_data=validation_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        validation_steps=validation_steps,
        verbose=2)

#### Transferencia del aprendizaje: Modelos ya implementados
- densenet module: DenseNet models for Keras.
- efficientnet module: EfficientNet models for Keras.
- inception_resnet_v2 module: Inception-ResNet V2 model for Keras.
- inception_v3 module: Inception V3 model for Keras.
- mobilenet module: MobileNet v1 models for Keras.
- mobilenet_v2 module: MobileNet v2 models for Keras.
- mobilenet_v3 module: MobileNet v3 models for Keras.
- nasnet module: NASNet-A models for Keras.
- resnet module: ResNet models for Keras.
- resnet_v2 module: ResNet v2 models for Keras.
- vgg16 module: VGG16 model for Keras.
- vgg19 module: VGG19 model for Keras.
- xception module: Xception V1 model for Keras.

![](img/ut06_39.png)

### Clasificación y localización de objetos
La localización de objetos en una imagen puede entenderse como un problema de regresión.

Para predecir el marco que encierra el objeto, la aproximación más general es intentar añadir el centro del objeto
y la anchura y altura de la caja que lo encierra.

Esta aproximación hace que haya que predecir cuatro números más, pero que no implica un cambio importante,
simplemente hay que añadir una capa Densa con cuatro unidades entrenadas con MSE

El problema se plantea a la hora de crear los datos de entrenamiento: tenemos muchos datos con etiquetas, pero hay
muy pocos datos con límites

In [ ]:
base_model = keras.applications.xception.Xception(
	 weights="imagenet",include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = keras.layers.Dense( n_classes, activation="softmax")(avg)
loc_output = keras.layers.Dense(4)(avg)
optimizer = keras.optimizers.SGD(Learning_rate=0.01, momentum=0.9,
                       nesterov=True, decay=0.001)
model = keras.Model(inputs=base_model.input,
	 outputs=[class_output, loc_output])
model.compile(loss=["sparse_categorical_crossentropy", "mse"],
	loss_weights=[0.8, 0.2], # depends on what you care most about
	optimizer=optimizer, metrics=["accuracy"])

### Redes Convolucionales Completas
La redes FCN se introdujeron para tareas de segmentación semántica. La idea es cambiar la última capa densa por una
covolucional, de tal manera que el tamaño del filtro tiene que ser igual al tamaño de la entrada de los mapas de
características y usar VALID para el padding, además el parámetro stride tiene que ser a 1.
La característica principal es que las FCN contiene solo capas convolucionales y permite que la red sea entrenada
en imágenes de cualquier tamaño.

La elección de la arquitectura de detección dependerá del problema.
- YOLO. Esta arquitectura se propuso para la detección de objetos en las imágenes y en sus diferentes versiones se han
ido mejorando paulatinamente.
- SSD
- Faster-RCNN

### Segmentación semántica
La segmentación semántica intenta clasificar los pixeles de una imagen de acuerdo a una clase, detectando la clase,
no la individualidad, con lo que si hay dos coches en la imagen juntos aparecerán los dos como coches y no como dos
objetos separados

Existen varias arquitecturas para este problema y las más sencillas se basan en una nueva capa Conv2DTranspose()

![](img/ut06_40.png)
